In [1]:
import json
import os
import pandas as pd
import ahpy
import process_engine.stats as pes

In [10]:
with open(os.path.join(os.getcwd(), 'raw_data', 'sample2.json'), 'r') as file:
    data = json.load(file)
idx, idx_alt = pes.generate_criteria_indecies(data)
criteria_length = len(idx)
print(idx)

{'building': 0, 'population': 1, 'elevation': 2, 'roads': 3, 'flood_area': 4, 'landuse': 5, 'slope': 6}


In [8]:
empty_matrix = pes.generate_decision_matrix(criteria_length)
pd.DataFrame(empty_matrix)

,0,1,2,3,4,5,6,7,8,9
0,0.665929,0.034919,0.608724,0.836764,0.251634,0.729484,0.492445,0.181093,0.488729,0.457226
1,0.602347,0.077842,0.802319,0.353230,0.920819,0.856847,0.456791,0.616838,0.563344,0.062165
2,0.081928,0.564654,0.639967,0.202457,0.186082,0.150954,0.099845,0.549378,0.341452,0.236283
3,0.862941,0.972890,0.357185,0.597934,0.702870,0.014857,0.266560,0.411852,0.988668,0.504678
4,0.880937,0.906590,0.359410,0.927797,0.292164,0.884984,0.170300,0.391197,0.095934,0.200208
5,0.805331,0.160638,0.815001,0.515523,0.046248,0.016768,0.617085,0.124794,0.719988,0.347003
6,0.060836,0.503264,0.023446,0.976495,0.857908,0.442561,0.019494,0.436116,0.893856,0.350643
7,0.677973,0.571255,0.928761,0.991600,0.881288,0.172122,0.067022,0.182650,0.566723,0.747790
8,0.657711,0.500071,0.926379,0.994988,0.136654,0.968529,0.290862,0.813639,0.118601,0.413960
9,0.040939,0.306079,0.282561,0.984435,0.063479,0.346249,0.295625,0.017121,0.095418,0.732466


In [9]:
filled_matrix = pes.populate_matrix(empty_matrix, data, idx)
pd.DataFrame(filled_matrix)

,0,1,2,3,4,5,6,7,8,9
0,1.000000,2.0,5.0,3.000000,1.000000,4.00,1.000000,2.000000,0.333333,5.00
1,0.500000,1.0,5.0,1.000000,1.000000,0.50,0.500000,2.000000,0.200000,2.00
2,0.200000,0.2,1.0,0.333333,0.250000,0.25,0.250000,0.250000,0.500000,0.25
3,0.333333,1.0,3.0,1.000000,0.500000,2.00,3.000000,1.000000,0.333333,0.25
4,1.000000,1.0,4.0,2.000000,1.000000,8.00,4.000000,2.000000,9.000000,7.00
5,0.250000,2.0,4.0,0.500000,0.125000,1.00,0.200000,4.000000,0.333333,4.00
6,1.000000,2.0,4.0,0.333333,0.250000,5.00,1.000000,3.000000,0.250000,6.00
7,0.500000,0.5,4.0,1.000000,0.500000,0.25,0.333333,0.182650,1.000000,3.00
8,3.000000,5.0,2.0,3.000000,0.111111,3.00,4.000000,1.000000,1.000000,4.00
9,0.200000,0.5,4.0,4.000000,0.142857,0.25,0.166667,0.333333,0.250000,1.00


In [10]:
pd.DataFrame(filled_matrix).to_csv('filled_matrix_sample_2.csv')

In [46]:
pd.DataFrame(filled_matrix)[1].sum()

np.float64(17.53333333333333)

In [47]:
weighted_matrix = pes.weighted_matrix(filled_matrix)
pd.DataFrame(weighted_matrix)

[27.         10.06666667 21.66666667  4.9047619   2.46666667 17.53333333
 20.80538506]


,0,1,2,3,4,5,6
0,0.037037,0.111111,0.111111,0.259259,0.185185,0.185185,0.111111
1,0.033113,0.099338,0.019868,0.298013,0.496689,0.033113,0.019868
2,0.015385,0.230769,0.046154,0.323077,0.230769,0.138462,0.015385
3,0.029126,0.067961,0.029126,0.203883,0.611650,0.029126,0.029126
4,0.081081,0.081081,0.081081,0.135135,0.405405,0.135135,0.081081
5,0.011407,0.171103,0.019011,0.399240,0.171103,0.057034,0.171103
6,0.016021,0.240322,0.144193,0.336451,0.240322,0.016021,0.006667


In [ ]:
pd.DataFrame(weighted_matrix).mean(axis=1)

0    0.142857
1    0.142857
2    0.142857
3    0.142857
4    0.142857
5    0.142857
6    0.142857
dtype: float64

In [49]:
weighted_criteria = pes.weighted_criteria(weighted_matrix)
pd.DataFrame(weighted_criteria)

,0
0,0.142857
1,0.142857
2,0.142857
3,0.142857
4,0.142857
5,0.142857
6,0.142857


In [ ]:
# print(weighted_criteria)
lambda_max = pes.lambda_max(filled_matrix, weighted_criteria)
ci = pes.consistency_index(lambda_max, criteria_length)
cr = pes.consistency_ratio(ci, pes.get_random_interval(criteria_length))
print(lambda_max, ci, cr)

27.000000000000004 3.333333333333334 2.5252525252525255


: 

In [11]:
cleaned = {}

for key, value in data.items():
    for subkey, subvalue in value.items():
        importancy = subvalue['importancy']
        scale = subvalue['scale']
        if (importancy == 'A'):
            cleaned[(key, subkey)] = scale
        else:
            cleaned[(key, subkey)] = pes.reciprocical_value(int(scale))

In [12]:
shelter = ahpy.Compare(name='shelter', comparisons=cleaned,
                       precision=3, random_index='saaty')

In [13]:
print(shelter.target_weights)

{'flood_area': np.float64(0.337), 'slope': np.float64(0.19), 'elevation': np.float64(0.183), 'roads': np.float64(0.163), 'building': np.float64(0.065), 'population': np.float64(0.032), 'landuse': np.float64(0.03)}


In [14]:
print(shelter.consistency_ratio)

0.175


In [15]:
shelter.report()

{'name': 'shelter',
 'global_weight': 1.0,
 'local_weight': 1.0,
 'target_weights': {'flood_area': np.float64(0.337),
  'slope': np.float64(0.19),
  'elevation': np.float64(0.183),
  'roads': np.float64(0.163),
  'building': np.float64(0.065),
  'population': np.float64(0.032),
  'landuse': np.float64(0.03)},
 'elements': {'global_weights': {'flood_area': np.float64(0.337),
   'slope': np.float64(0.19),
   'elevation': np.float64(0.183),
   'roads': np.float64(0.163),
   'building': np.float64(0.065),
   'population': np.float64(0.032),
   'landuse': np.float64(0.03)},
  'local_weights': {'flood_area': np.float64(0.337),
   'slope': np.float64(0.19),
   'elevation': np.float64(0.183),
   'roads': np.float64(0.163),
   'building': np.float64(0.065),
   'population': np.float64(0.032),
   'landuse': np.float64(0.03)},
  'consistency_ratio': np.float64(0.175)}}